From: https://langchain-ai.github.io/langgraph/tutorials/multi_agent/hierarchical_agent_teams/
Adapt to deepseek 

In [ ]:
import getpass
import os
from langchain_core.tools import tool
from langchain_deepseek import ChatDeepSeek

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

_set_if_undefined("DEEPSEEK_API_KEY")

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=10, 
)

def analyse_vendor(llm, question):
    return llm.invoke(question)

@tool
def analyse_vendor_tool(question: str) -> str:
    """Use this tool to answer decide how many units to buy."""
    return analyse_vendor(llm, question)

def analyse_demand(llm, question):
    return llm.invoke(question)

@tool
def analyse_demand_tool(question: str) -> str:
    """Use this tool to decide how many units to produce."""
    return analyse_vendor(llm, question)

In [41]:
from typing import List, Optional, Literal
from langchain_core.language_models.chat_models import BaseChatModel
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command
from langchain_core.messages import HumanMessage, trim_messages
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

class State(MessagesState):
    next: str
    visited: List[str]

def make_supervisor_node(llm: BaseChatModel, members: list[str]) -> str:
    options = ["FINISH"] + members
    system_prompt = (
        "You are a supervisor tasked with managing a conversation between the"
        f" following companies: {members}. Your job is to coordinate their work.\n"
        " Rules:\n"
        " 1. You must route to EACH company EXACTLY ONCE before finishing\n"
        " 2. Track which companies have already participated\n"
        " 3. Only choose FINISH when ALL companies have participated\n"
        " 4. Choose the next company that hasn't participated yet"
    )
    class Router(TypedDict):
        """Worker to route to next until all have participated."""
        next: Literal[*options]

    def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
        """An LLM-based router that ensures all agents participate."""
        # Initialize visited list if not present
        if "visited" not in state:
            state["visited"] = []

        messages = [
            {"role": "system", "content": system_prompt + f"\nAlready visited: {state['visited']}"},
        ] + state["messages"]

        response = llm.with_structured_output(Router).invoke(messages)
        goto = response["next"]

        # If all members have been visited, then we can finish
        if len(state["visited"]) >= len(members):
            goto = "FINISH"

        if goto == "FINISH":
            goto = END
        else:
            # Add the next company to visited list
            state["visited"] = state["visited"] + [goto]

        return Command(goto=goto, update={"next": goto, "visited": state["visited"]})
    return supervisor_node

In [51]:
companyA_agent = create_react_agent(llm,tools=[analisys_tool])
def companyA_node(state: State) -> Command[Literal["supervisor"]]:
    result = companyA_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="companyA")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )

companyB_agent = create_react_agent(llm,tools=[analisys_tool])
def companyB_node(state: State) -> Command[Literal["supervisor"]]:
    result = companyB_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="companyB")
            ]
        },
        goto="supervisor",
    )

research_supervisor_node = make_supervisor_node(llm, ["companyA", "companyB"])

research_builder = StateGraph(State)
research_builder.add_node("supervisor", research_supervisor_node)
research_builder.add_node("companyA", companyA_node)
research_builder.add_node("companyB", companyB_node)
research_builder.add_edge(START, "supervisor")

research_graph = research_builder.compile()

initial_message = HumanMessage(content="how many OLT will you buy")
for s in research_graph.stream(
    {"messages": [initial_message]},
    {"recursion_limit": 10},
):
    print(s)
    print("---")

{'supervisor': {'next': 'companyA', 'visited': ['companyA']}}
---
{'companyA': {'messages': [HumanMessage(content='The number of OLTs (Optical Line Terminals) you should buy depends on several factors, including:\n\n1. **Network Size & Coverage** – How many subscribers or endpoints (ONUs) need to be served?\n2. **Port Density** – How many PON ports (GPON, XGS-PON, etc.) does each OLT support?\n3. **Redundancy & Scalability** – Do you need backup OLTs for high availability?\n4. **Future Growth** – Are you planning to expand soon?\n\n### General Estimation:\n- A single OLT chassis can support **hundreds to thousands of ONUs** (depending on the model and split ratio).\n- Example: If you have **1,000 subscribers** and use a **1:64 split ratio**, you might need:\n  - **~16 PON ports** (1,000 ÷ 64 ≈ 15.6)\n  - **1-2 OLTs** (depending on redundancy).\n\nWould you like a more precise calculation based on your specific requirements (subscribers, split ratio, vendor model, etc.)? Let me know!', 

In [48]:
ISP_agent = create_react_agent(
    llm,tools=[analisys_tool], 
    prompt=(
        "You are ISP and decide how many OLT buy"
    ),
)

def ISP_A_node(state: State) -> Command[Literal["supervisor"]]:
    result = ISP_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="ISP_A")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )

def ISP_B_node(state: State) -> Command[Literal["supervisor"]]:
    result = ISP_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="ISP_B")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )

companyA_supervisor_node = make_supervisor_node(
    llm, ["ISP_A","ISP_B"]
)

In [49]:
paper_writing_builder = StateGraph(State)
paper_writing_builder.add_node("supervisor", companyA_supervisor_node)
paper_writing_builder.add_node("ISP_A", ISP_A_node)
paper_writing_builder.add_node("ISP_B", ISP_B_node)

paper_writing_builder.add_edge(START, "supervisor")
paper_writing_graph = paper_writing_builder.compile()

In [ ]:
seen_states = set()
for s in paper_writing_graph.stream(
    {
        "messages": [
            (
                "user",
                "How many OLt will you buy",
            )
        ]
    },
    {"recursion_limit": 100},
):
    print(s)
    print("---")

{'supervisor': {'next': 'ISP_A', 'visited': ['ISP_A']}}
---
{'ISP_A': {'messages': [HumanMessage(content="The number of OLT (Optical Line Terminal) units you should purchase depends on several factors, including your subscriber base, network architecture, and future growth plans. Here's a summary to guide your decision:\n\n### **Key Considerations:**\n1. **Subscriber Demand & Density**  \n   - Estimate the number of subscribers per OLT port (typically 32–64 users per PON port, depending on the split ratio).  \n   - Example: For 1,000 subscribers with a 1:32 split ratio, you'd need about 32 PON ports (≈ 1–2 OLTs, assuming 16–32 ports per OLT).  \n\n2. **Bandwidth Requirements**  \n   - Higher bandwidth services (e.g., 1Gbps+ plans) may require fewer users per PON port to avoid congestion.  \n\n3. **Redundancy & Future Growth**  \n   - Plan for **N+1 redundancy** (an extra OLT for failover).  \n   - Allocate **20–30% capacity** for growth over 3–5 years.  \n\n4. **Geographical Distributi

In [55]:
from langchain_core.messages import BaseMessage
teams_supervisor_node = make_supervisor_node(llm, ["call_companyA", "call_companyB"])

In [ ]:
def call_companyA(state: State) -> Command[Literal["supervisor"]]:
    response = research_graph.invoke({"messages": state["messages"][-1]})
    return Command(
        update={
            "messages": [
                HumanMessage(
                    content=response["messages"][-1].content, name="companyA"
                )
            ]
        },
        goto="supervisor",
    )


def call_companyB(state: State) -> Command[Literal["supervisor"]]:
    response = paper_writing_graph.invoke({"messages": state["messages"][-1]})
    return Command(
        update={
            "messages": [
                HumanMessage(
                    content=response["messages"][-1].content, name="companyB"
                )
            ]
        },
        goto="supervisor",
    )


# Define the graph.
super_builder = StateGraph(State)
super_builder.add_node("supervisor", teams_supervisor_node)
super_builder.add_node("call_companyA", call_companyA)
super_builder.add_node("call_companyB", call_companyB)

super_builder.add_edge(START, "supervisor")
super_graph = super_builder.compile()

In [57]:
for s in super_graph.stream(
    {
        "messages": [
            ("user", "You are in interent industry decide how many OLT buy")
        ],
    },
    {"recursion_limit": 150},
):
    print(s)
    print("---")

{'supervisor': {'next': 'call_companyA', 'visited': ['call_companyA']}}
---
{'call_companyA': {'messages': [HumanMessage(content="For your deployment serving **20,000 users** with **GPON (1:64 split ratio)** and **16-port OLTs**, including redundancy, you should purchase **21 OLT units**. Here's the breakdown:\n\n### **Calculation:**\n1. **ONUs per OLT Port:**  \n   - Each OLT port serves **64 ONUs** (split ratio 1:64).  \n   - A **16-port OLT** can support:  \n     \\[\n     16 \\text{ ports} \\times 64 \\text{ ONUs/port} = 1,024 \\text{ ONUs per OLT}\n     \\]\n\n2. **OLTs Needed (Without Redundancy):**  \n   \\[\n   \\frac{20,000 \\text{ users}}{1,024 \\text{ ONUs per OLT}} \\approx 19.53 \\text{ OLTs}\n   \\]  \n   - Round up to **20 OLTs** (serving 20,480 ONUs, slightly over-provisioned).\n\n3. **Include Redundancy (N+1):**  \n   - Add **1 redundant OLT** for failover.  \n   - **Total OLTs:** **21**.\n\n### **Summary Table:**\n| **Parameter**         | **Value** |\n|--------------